# w_01 | `pandas plot()`

* Este notebook describe la funcionalidad del método `plot()` de `DataFrame` y `Series`.
* Además se cubre backends alternativos, en particular `Plotly Express`

* Primero traemos un dataset.

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY.csv"
data = pd.read_csv(url, index_col=0, parse_dates=True, dayfirst=True)
data.head()

In [ ]:
data.info()

* Como vimos, el backend por defecto es Matplotlib.

In [ ]:
pd.options.plotting.backend

* `plot()` sin argumentos grafica líneas.
    * Cada columna es una línea.
    * El índice se usa para el eje de las x.

In [ ]:
data.plot()

In [ ]:
data.reset_index(drop=True).plot()

* Se puede especificar una columna distinta que el `index` para usar como eje x.

In [ ]:
data.plot(x="dia")

* Se puede especificar solo algunas columnas para graficar.

In [ ]:
data.plot(y=["cantFallecidos", "cantCTI", "cantCI"])

* Notar que esto es lo mismo que slicear el df previamente.

In [ ]:
data[["cantFallecidos", "cantCTI", "cantCI"]].plot()

* El parámetro `subplots` es particularmente útil.
    * Crea un gráfico por cada columna.
* Combinar con `sharex` o `sharey` logra aún mejores resultados.
* Si no se define `layout`, apila los gráficos verticalmente.
    * `layout` acepta un `tuple` de `(filas, columnas)`.

In [ ]:
data.loc[:, data.columns.str.contains("acum")].plot(subplots=True,
                                                    sharex=True, layout=(2, 2))

* La anterior quedó un poco difícil de ver.
* Para eso podemos definir el `figsize` como un `tuple`.

In [ ]:
data.loc[:, data.columns.str.contains("acum")].plot(subplots=True, sharex=True,
                                                    layout=(2, 2), figsize=(8, 5))

* Podemos definir un título para el gráfico con `title`.

In [ ]:
data.plot(y=["cantTest", "cantCasosNuevosAjustado", "Positividad"],
          subplots=True, layout=(1, 3), figsize=(10, 4), title="Tests y positividad")

* La leyenda de los ejes se puede modificar con `xlabel` y `ylabel`.

In [ ]:
data.plot(y=["cantTest", "cantCasosNuevosAjustado", "Positividad"],
          subplots=True, layout=(1, 3), figsize=(10, 4), title="Tests y positividad", xlabel="")

* `colormap` define los colores de los gráficos.
* Ver las opciones en la [documentación de MPL](https://matplotlib.org/stable/gallery/color/colormap_reference.html).
* Creamos una variable llamada 'CFR' que es "Case Fatality Rate" o "Tasa de Letalidad". Se multiplica por 100 para obtener el porcentaje.
* `data.rolling(7).mean()` En esta línea de código, se calcula la media móvil con ventana de tamaño 7 para todas las columnas en el DataFrame data. La media móvil es una técnica que suaviza los datos y ayuda a visualizar tendencias a largo plazo. Esto calcula la media de los últimos 7 días para cada fila de datos.
* `data.loc[data.index > "2021-01-01": `: Aquí, se filtran las filas en el DataFrame data y se seleccionan solo las filas con índices (fechas) mayores a "2021-01-01".

In [ ]:
data["CFR"] = (data["cantFallecidos"]
               / data["cantCasosNuevosAjustado"].shift(14)) * 100

data.rolling(7).mean().loc[data.index > "2021-01-01"].plot(y=["cantCasosNuevosAjustado", "cantFallecidos", "CFR"],
                                                           figsize=(10, 5), colormap="Dark2", subplots=True,
                                                           layout=(1, 3))

* También es posible definir un estilo para todos los gráficos.
* Ver galería de estilos en [la documentación](https://matplotlib.org/stable/gallery/style_sheets/style_sheets_reference.html).
* Notar que es la primera vez que necesitamos importar Matplotlib directamente.
    * El statement clásico es `import matplotlib.pyplot as plt`

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("fivethirtyeight")

data.rolling(7).mean().loc[data.index > "2021-01-01"].plot(y=["cantCasosNuevosAjustado", "cantFallecidos", "CFR"],
                                                           figsize=(10, 5), subplots=True,
                                                           layout=(1, 3))

* Uno de los problemas de MPL es que las modificaciones que necesitemos hacer más allá del setup básico son bastante verbosas.
* Por ejemplo, cambiar la ubicación de la leyenda de esos 3 gráficos requiere crear subplots de ante mano, ir agregando los plots individuales y cambiando las leyendas en un loop.
* `bbox_to_anchor=(0.5, -0.3)` : Este argumento define la posición de la leyenda con respecto a los ejes del gráfico. El valor (0.5, -0.3) indica que la leyenda se colocará en la posición horizontal central (0.5) y en la posición vertical -0.3 unidades por debajo del gráfico. Los valores en el eje vertical son negativos porque -0.3 coloca la leyenda debajo del gráfico principal.

* `loc="lower center"`: Este argumento especifica la posición relativa de la leyenda dentro de la caja definida por bbox_to_anchor. "lower center" significa que la leyenda se colocará en la parte inferior central de la caja. Las opciones disponibles para loc pueden ser combinaciones de palabras clave que indiquen la posición en la caja, como "upper left", "upper right", "lower left", "lower right", "center", "center left", "center right", etc.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(10, 5))

for ax, col in zip(axes, ["cantCasosNuevosAjustado", "cantFallecidos", "CFR"]):
    data.rolling(7).mean().loc[data.index > "2021-01-01", col].plot(ax=ax, xlabel="")
    ax.legend(bbox_to_anchor=(0.5, -0.3), loc="lower center")
plt.show()

* Notar que el estilo elegido quedó fijado para todas las figuras.
* Se puede usar un estilo para un bloque de código particular usando un context manager.

In [ ]:
with plt.style.context("grayscale"):
    data.plot(y="cantCTI")
plt.show()

* Hasta ahora vimos solo el gráfico por defecto que es de líneas, pero pandas admite varios otros.
* El parámetro `kind` define el tipo de gráfico.
* Notar el uso de `ravel()`, un método de numpy, para transformar una matriz de 2x2 en una lista y poder iterar.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 6))
for ax, kind in zip(axes.ravel(), ["hist", "area", "kde", "scatter"]):
    if kind in ["hexbin", "scatter"]:
        data.plot(y="cantFallecidos", x="cantCTI", ax=ax, kind=kind, title=kind)
    else:
        data.plot(y=["cantFallecidos", "cantCTI"], ax=ax, kind=kind, stacked=False, xlabel="", title=kind)
fig.tight_layout()
fig.suptitle("Varios kind")
plt.show()

* Cuidado con `kind="bar"`, porque asume que los datos son categóricos.

In [ ]:
data.reset_index().plot(kind="bar", y="cantCTI", x="fecha")

* `plt.bar()` no tiene ese problema, pero no formatea las fechas tan bien.

In [ ]:
plt.bar(height=data.loc[:, "cantCTI"], x=data.index)
plt.xticks(rotation=90)
plt.show()

* Formatearlas bien es algo engorroso.
* `mdates` de `matplotlib.dates` proporciona funciones para trabajar con fechas y horas en las visualizaciones.
* `_, ax = plt.subplots()` Crea un nuevo conjunto de ejes (subplots) utilizando `plt.subplots()`. El subguión `_` es una convención para ignorar el valor que devuelve `plt.subplots()` que representa la figura. La variable `ax` se utiliza para referenciar los ejes del gráfico.
* `locator = mdates.AutoDateLocator(minticks=3, maxticks=7)`: se crea un localizador de fechas automático utilizando `mdates.AutoDateLocator()`. Este localizador determinará automáticamente la ubicación y cantidad de marcas en el eje X para las fechas. Se establecen los argumentos `minticks=3` y `maxticks=7`, lo que indica que se deben mostrar entre 3 y 7 marcas de fecha en el eje X.

* `formatter = mdates.ConciseDateFormatter(locator)`: Se crea un formateador de fechas conciso utilizando `mdates.ConciseDateFormatter()`. Este formateador tomará las marcas de fecha generadas por el localizador y las formateará en un formato más conciso y legible.

* `ax.xaxis.set_major_locator(locator)`: Se establece el localizador de fechas creado anteriormente en el eje X utilizando `ax.xaxis.set_major_locator()`. Esto asegura que las marcas de fecha se posicionen correctamente en el eje X.
* `ax.xaxis.set_major_formatter(formatter)`: Se establece el formateador de fechas creado anteriormente en el eje X utilizando `ax.xaxis.set_major_formatter()`. Esto garantiza que las fechas se muestren en el formato conciso y legible proporcionado por el formateador.

In [ ]:
import matplotlib.dates as mdates

_, ax = plt.subplots()
ax.bar(height=data.loc[:, "cantCTI"], x=data.index)
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)
plt.show()

* Podemos cambiar el backend de pandas si lo tenemos instalado.

In [ ]:
pd.options.plotting.backend = "plotly"

data.plot(y="acumCasos", kind="bar")

* Pero no todas las opciones funcionan.
* el método `bar()` de `plotly` no acepta el argumento `figsize`, que es un argumento específico de `matplotlib`

In [ ]:
import traceback

try:
    data.plot(y="acumCasos", kind="bar", figsize=(6, 4))
except TypeError:
    traceback.print_exc()

* Por el contrario, pandas admite que pasemos `kwargs` (keyword arguments) que son transferidos a la función/método del paquete correspondiente.
    * En este caso pasamos `width`, `height` y `template`.

In [ ]:
data.plot(y="acumCasos", kind="bar", width=600, height=400, template="seaborn")

* En estos casos es mejor ir a ver la documentación del backend que la de pandas directamente.

In [ ]:
deptos_url = "https://raw.githubusercontent.com/GUIAD-COVID/datos-y-visualizaciones-GUIAD/master/datos/estadisticasUY_porDepto_detalle.csv"
deptos = pd.read_csv(deptos_url, index_col=0, parse_dates=True, dayfirst=True)
deptos.head()

In [ ]:
deptos.plot(y="enCurso", color="departamento", kind="area")

***Nota***:  recuerda que `.isin` es una función de la biblioteca pandas en Python que se utiliza para filtrar un DataFrame o una Serie basándose en una condición de pertenencia. Su objetivo es seleccionar filas que contengan valores específicos en una columna determinada.

La sintaxis general del método `.isin()` es la siguiente:

`dataframe['columna'].isin(valores_a_buscar)`



In [ ]:
fig = deptos.loc[deptos["departamento"]
                 .isin(["Rocha(UY-RO)", "Artigas(UY-AR)",
                        "Maldonado(UY-MA)", "Rivera(UY-RV)"])].plot(y="enCurso", facet_row="departamento",
                                                  kind="area", title="Casos en curso", width=600, height=800)
fig.update_yaxes(matches=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()